In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
wang_db = pd.read_csv("../data/reference/13321_2020_470_MOESM1_ESM.csv")
wang_props = ["RBN", "Dot", "Mass", "Match", "number_of_atoms", "Cos", "R.match", "RBF", "PHI"]
# Assuming `df` is your DataFrame
corr_wang = wang_db[wang_props].corr().round(2)

plt.figure(figsize=(10, 8))
sns.heatmap(corr_wang, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
# plt.title('Pearson Correlations')
plt.show()

In [ ]:
wang_colwise_means = wang_db[wang_props].mean()
our_means = df[properties].mean()